<a href="https://colab.research.google.com/github/mateuszCh77/MEDICA/blob/main/Implementacja_kodu_lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# Blok 1: Dodawanie Nowego Pacjenta

# --- Importy ---
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML as IPHTML # Zmieniono nazwę importu display.HTML
import hashlib # Do (prostego) hashowania hasła
import datetime # Do obsługi daty

# --- Konfiguracja Bazy Danych ---
DB_NAME = 'medica_plus.db'

def get_db_connection():
    """Nawiązuje połączenie z bazą danych SQLite."""
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row # Zwraca wyniki jako obiekty podobne do słowników
    return conn

def create_patients_table_if_not_exists():
    """Tworzy tabelę 'patients', jeśli nie istnieje."""
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS patients (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                first_name TEXT NOT NULL,
                last_name TEXT NOT NULL,
                pesel TEXT UNIQUE NOT NULL,
                dob TEXT,
                phone TEXT,
                email TEXT,
                address TEXT,
                contact_person TEXT,
                blood_group TEXT,
                allergies TEXT,
                chronic_diseases TEXT,
                current_meds TEXT,
                important_notes TEXT,
                login TEXT UNIQUE NOT NULL,
                password_hash TEXT NOT NULL, -- Zapisujemy hash hasła
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        conn.commit()
    except sqlite3.Error as e:
        print(f"Błąd podczas tworzenia/sprawdzania tabeli: {e}")
    finally:
        conn.close()

# --- Funkcje Pomocnicze ---
def hash_password(password):
    """Proste hashowanie hasła (SHA-256). W realnym systemie użyj bcrypt/argon2!"""
    return hashlib.sha256(password.encode()).hexdigest()

# --- Logika Backend - Dodawanie Pacjenta ---
def add_patient(data):
    """Dodaje nowego pacjenta do bazy danych."""
    required_fields = ['first_name', 'last_name', 'pesel', 'login', 'password']
    # Używamy description do walidacji, usuwamy '*'
    missing_fields = [field for field in required_fields if not data.get(field)]
    if missing_fields:
        # Mapowanie nazw technicznych na nazwy z opisów pól
        field_map = {
            'first_name': 'Imię', 'last_name': 'Nazwisko', 'pesel': 'PESEL',
            'login': 'Login', 'password': 'Hasło'
        }
        missing_labels = [field_map.get(f, f) for f in missing_fields]
        return False, f"Wypełnij wymagane pola: {', '.join(missing_labels)}."


    # Prosta walidacja PESEL (długość i cyfry)
    if len(data['pesel']) != 11 or not data['pesel'].isdigit():
            return False, "Błąd: PESEL musi składać się z 11 cyfr."

    # Walidacja hasła (minimalna długość)
    if len(data['password']) < 8:
         return False, "Błąd: Hasło musi mieć co najmniej 8 znaków."

    hashed_pw = hash_password(data['password'])
    sql = """
        INSERT INTO patients (
            first_name, last_name, pesel, dob, phone, email, address,
            contact_person, blood_group, allergies, chronic_diseases,
            current_meds, important_notes, login, password_hash
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    # Konwertuj datę na string YYYY-MM-DD lub zostaw None
    dob_str = data.get('dob').strftime('%Y-%m-%d') if data.get('dob') else None

    values = (
        data.get('first_name'), data.get('last_name'), data.get('pesel'),
        dob_str, data.get('phone'), data.get('email'), data.get('address'),
        data.get('contact_person'), data.get('blood_group'), data.get('allergies'),
        data.get('chronic_diseases'), data.get('current_meds'), data.get('important_notes'),
        data.get('login'), hashed_pw
    )

    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(sql, values)
        conn.commit()
        return True, f"Pacjent {data['first_name']} {data['last_name']} dodany pomyślnie (ID: {cursor.lastrowid})."
    except sqlite3.IntegrityError as e:
        error_msg = str(e).lower()
        if 'patients.pesel' in error_msg: # SQLite zwraca nazwę tabeli i kolumny
            return False, "Błąd: Pacjent z tym numerem PESEL już istnieje."
        elif 'patients.login' in error_msg:
             return False, "Błąd: Podany login jest już zajęty."
        else:
            return False, f"Błąd unikalności bazy danych: {e}"
    except sqlite3.Error as e:
        return False, f"Błąd bazy danych: {e}"
    finally:
        conn.close()

# --- Interfejs Użytkownika (ipywidgets) ze stylizacją ---

# Upewnij się, że tabela istnieje przed stworzeniem widżetów
create_patients_table_if_not_exists()

# --- Wstrzykiwanie CSS ---
# Używamy klas CSS z ipywidgets (można je znaleźć przez inspekcję elementu w przeglądarce)
# oraz dodajemy własne klasy (np. form-field, form-grid-item) dla lepszego targetowania
custom_css = widgets.HTML("""
<style>
    /* Kontener główny - odtworzenie .form-container */
    .custom-form-container {
        background-color: #ffffff;
        padding: 30px;
        border-radius: 8px;
        box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        border-top: 5px solid #0056b3;
        font-family: sans-serif; /* Stosujemy font do kontenera */
    }

    /* Nagłówek główny - odtworzenie h2 */
    .custom-form-container > .widget-html h2 {
        color: #0056b3;
        border-bottom: 1px solid #eeeeee;
        padding-bottom: 10px;
        margin-top: 0; /* Reset margin */
        margin-bottom: 25px;
        text-align: center;
        font-size: 1.5em; /* Dopasowanie rozmiaru */
        font-weight: bold;
    }

    /* Kontener sekcji - odtworzenie .form-section */
    .custom-form-section {
        margin-bottom: 30px !important; /* Używamy !important by nadpisać domyślne marginesy VBox */
        padding: 20px !important;
        border: 1px solid #e0e0e0 !important;
        border-radius: 5px !important;
        background-color: #fdfdfd !important;
    }

    /* Nagłówek sekcji - odtworzenie .form-section h3 */
    .custom-form-section > .widget-html h3 {
        color: #333;
        margin-top: 0;
        margin-bottom: 20px;
        font-size: 1.1em;
        border-bottom: 1px solid #eee;
        padding-bottom: 5px;
        font-weight: bold;
    }

    /* Siatka pól - GridBox sam tworzy siatkę, ustawiamy tylko odstępy */
    .custom-gridbox {
        /* Atrybuty grid są w layout GridBox, tu tylko ew. dodatki */
    }

    /* Pole formularza (kontener dla etykiety i inputu) - odtworzenie .form-field */
    .form-field {
        display: flex;
        flex-direction: column;
        margin-bottom: 5px; /* Dodajemy mały margines dla lepszego wyglądu */
    }

    /* Etykieta pola - stylizujemy .widget-label */
    .form-field .widget-label {
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 0.9em;
        color: #555;
        /* Domyślne wyrównanie etykiet w ipywidgets jest inne, zachowujemy je */
        /* order: -1; /* Jeśli chcemy etykietę zawsze na górze */
    }

    /* Inputy, Textarea, DatePicker - odtworzenie stylów inputów */
    .form-field .widget-text input,
    .form-field .widget-password input,
    .form-field .widget-numeric-text input, /* Dla potencjalnych pól numerycznych */
    .form-field .widget-datepicker input[type="text"], /* Celujemy w input DatePicker */
    .form-field .widget-textarea textarea {
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 4px;
        font-size: 1em;
        width: 100%; /* Chcemy by zajmowały całą komórkę siatki */
        box-sizing: border-box;
        background-color: #fff; /* Ensure background is white */
    }

    .form-field .widget-textarea textarea {
        min-height: 80px;
        resize: vertical;
    }

    /* Efekt focus dla inputów */
    .form-field .widget-text input:focus,
    .form-field .widget-password input:focus,
    .form-field .widget-numeric-text input:focus,
    .form-field .widget-datepicker input[type="text"]:focus,
    .form-field .widget-textarea textarea:focus {
        border-color: #0056b3;
        outline: none;
        box-shadow: 0 0 0 2px rgba(0, 86, 179, 0.2);
    }

    /* Kontener akcji - odtworzenie .form-actions */
    .custom-form-actions {
        display: flex; /* Używamy flexbox do wyrównania */
        justify-content: flex-end; /* Wyrównanie do prawej */
        margin-top: 30px;
    }

    /* Przycisk - odtworzenie .form-actions button */
    .custom-form-actions .widget-button button {
        background-color: #0056b3;
        color: white;
        padding: 12px 25px;
        border: none;
        border-radius: 5px;
        font-size: 1em;
        cursor: pointer;
        transition: background-color 0.3s ease;
    }

    .custom-form-actions .widget-button button:hover {
        background-color: #004494;
    }

    /* Komunikaty wyjściowe */
    .widget-output {
      margin-top: 15px;
      padding: 10px;
      border-radius: 4px;
    }
    .widget-output pre { /* Celujemy w <pre> generowane przez print() w Output */
        font-size: 1em;
        white-space: pre-wrap; /* Zawijanie tekstu */
        margin: 0;
    }
     /* Dodatkowe klasy dla sukcesu/błędu (ustawiane ręcznie) */
    .output-success {
        background-color: #e6ffed;
        border: 1px solid #a3e9a4;
        color: #1f7c3a;
    }
    .output-error {
        background-color: #ffebee;
        border: 1px solid #ffcdd2;
        color: #c62828;
    }

</style>
""")

# --- Definicja Widgetów ---

# Funkcja pomocnicza do tworzenia pola (Etykieta + Widget) dla spójności
def create_form_field(widget, label_text):
    # Usuwamy '*' z description widgetu, bo etykietę tworzymy sami
    widget.description = "" # Nie używamy wbudowanej etykiety, by mieć pełną kontrolę
    label = widgets.Label(label_text, layout=widgets.Layout(margin='0 0 5px 0'))
    # Zwracamy VBox, który symuluje .form-field
    return widgets.VBox([label, widget], layout=widgets.Layout(width='auto'), add_class='form-field') # auto width dla flex/grid item


# Sekcja Dane Osobowe
w_first_name = widgets.Text(placeholder="Jan")
w_last_name = widgets.Text(placeholder="Kowalski")
w_pesel = widgets.Text(placeholder="12345678901")
w_dob = widgets.DatePicker(disabled=False)
w_phone = widgets.Text(placeholder="+48 123 456 789")
w_email = widgets.Text(placeholder="jan.kowalski@example.com")
w_address = widgets.Textarea(placeholder="ul. Zdrowa 1, 00-001 Warszawa", layout=widgets.Layout(height='80px'))
w_contact_person = widgets.Text(placeholder="Imię, nazwisko, numer telefonu")

# Układ siatki dla danych osobowych
personal_grid = widgets.GridBox(children=[
    create_form_field(w_first_name, "Imię:*"),
    create_form_field(w_last_name, "Nazwisko:*"),
    create_form_field(w_pesel, "PESEL:*"),
    create_form_field(w_dob, "Data urodzenia:"),
    create_form_field(w_phone, "Telefon:"),
    create_form_field(w_email, "Email:"),
    create_form_field(w_address, "Adres:"), # Pole adresu
    create_form_field(w_contact_person, "Osoba kontaktowa (opcjonalnie):") # Pole osoby kontaktowej
], layout=widgets.Layout(
    grid_template_columns="repeat(auto-fit, minmax(250px, 1fr))", # Responsive grid
    grid_gap='15px',
    width='100%' # Grid zajmuje całą szerokość sekcji
))
# Ustawienie rozpiętości dla adresu i osoby kontaktowej (w nowej wersji ipywidgets to działa)
# Sprawdź wersję ipywidgets. Jeśli to nie działa, alternatywą jest użycie VBox i HBox.
try:
    w_address.layout.grid_column = '1 / -1' # Span all columns
    w_contact_person.layout.grid_column = '1 / -1' # Span all columns
except AttributeError:
    # Fallback dla starszych wersji - użyjemy VBox/HBox zamiast GridBox lub zaakceptujemy brak span
    print("Ostrzeżenie: Twoja wersja ipywidgets może nie wspierać `layout.grid_column`. Rozważ aktualizację lub alternatywny układ.")
    # Tutaj można by zaimplementować układ z VBox/HBox jako alternatywę

personal_data_section = widgets.VBox([
    widgets.HTML("<h3>Dane Osobowe</h3>"),
    personal_grid
], add_class='custom-form-section') # Dodajemy klasę CSS

# Sekcja Informacje Medyczne
w_blood_group = widgets.Text(placeholder="np. A Rh+")
w_allergies = widgets.Textarea(placeholder="np. Penicylina, kurz, pyłki traw", layout=widgets.Layout(height='80px'))
w_chronic = widgets.Textarea(placeholder="np. Nadciśnienie tętnicze, Cukrzyca typu 2", layout=widgets.Layout(height='80px'))
w_meds = widgets.Textarea(placeholder="np. Metformina 500mg, Euthyrox 50mcg", layout=widgets.Layout(height='80px'))
w_notes = widgets.Textarea(placeholder="Dodatkowe informacje istotne dla leczenia", layout=widgets.Layout(height='80px'))

# Układ siatki dla danych medycznych
medical_grid = widgets.GridBox(children=[
     create_form_field(w_blood_group, "Grupa krwi:"),
     create_form_field(w_allergies, "Alergie:"), # Pole alergii
     create_form_field(w_chronic, "Choroby przewlekłe:"), # Pole chorób
     create_form_field(w_meds, "Stałe leki:"), # Pole leków
     create_form_field(w_notes, "Ważne uwagi:") # Pole uwag
], layout=widgets.Layout(
    grid_template_columns="repeat(auto-fit, minmax(250px, 1fr))",
    grid_gap='15px',
    width='100%'
))
# Ustawienie rozpiętości dla pól textarea
try:
    w_allergies.layout.grid_column = '1 / -1'
    w_chronic.layout.grid_column = '1 / -1'
    w_meds.layout.grid_column = '1 / -1'
    w_notes.layout.grid_column = '1 / -1'
except AttributeError:
    pass # Ostrzeżenie wyświetlone wcześniej

medical_data_section = widgets.VBox([
    widgets.HTML("<h3>Podstawowe Informacje Medyczne</h3>"),
    medical_grid
], add_class='custom-form-section') # Dodajemy klasę CSS

# Sekcja Dane Logowania
w_login = widgets.Text(placeholder="Wprowadź login dla pacjenta")
w_password = widgets.Password(placeholder="Wprowadź hasło dla pacjenta")

# Układ siatki dla danych logowania
login_grid = widgets.GridBox(children=[
    create_form_field(w_login, "Login (np. PESEL lub email):*"),
    create_form_field(w_password, "Hasło:*")
], layout=widgets.Layout(
    grid_template_columns="repeat(auto-fit, minmax(250px, 1fr))",
    grid_gap='15px',
    width='100%'
))

login_data_section = widgets.VBox([
    widgets.HTML("<h3>Dane Logowania do Systemu Pacjenta</h3>"),
    login_grid
], add_class='custom-form-section') # Dodajemy klasę CSS

# Przycisk i Pole Wyniku
add_patient_button = widgets.Button(description="Dodaj Pacjenta", icon='plus', tooltip="Kliknij, aby zapisać dane nowego pacjenta")
# Kontener dla przycisku, aby zastosować styl .form-actions
button_container = widgets.HBox([add_patient_button], layout=widgets.Layout(justify_content='flex-end'), add_class='custom-form-actions')

add_patient_output = widgets.Output() # Miejsce na komunikaty

# --- Logika Przycisku Dodawania Pacjenta (Callback) ---
def on_add_patient_button_click(b):
    # Zbierz dane z formularza
    patient_data = {
        # Usuwamy strip() z pól numerycznych/haseł jak PESEL i password
        "first_name": w_first_name.value.strip(),
        "last_name": w_last_name.value.strip(),
        "pesel": w_pesel.value, # PESEL bez strip
        "dob": w_dob.value, # Przekaż obiekt daty lub None
        "phone": w_phone.value.strip(),
        "email": w_email.value.strip(),
        "address": w_address.value.strip(),
        "contact_person": w_contact_person.value.strip(),
        "blood_group": w_blood_group.value.strip(),
        "allergies": w_allergies.value.strip(),
        "chronic_diseases": w_chronic.value.strip(),
        "current_meds": w_meds.value.strip(),
        "important_notes": w_notes.value.strip(),
        "login": w_login.value.strip(),
        "password": w_password.value # Hasło bez strip
    }

    # Wywołaj funkcję backendową i wyświetl wynik w Output
    with add_patient_output:
        clear_output(wait=True) # Wyczyść poprzednie komunikaty
        success, message = add_patient(patient_data)
        if success:
            # Dodaj klasę CSS do output dla stylizacji sukcesu
            add_patient_output.add_class("output-success")
            add_patient_output.remove_class("output-error") # Usuń klasę błędu jeśli była
            print(f"\u2705 {message}") # Zielony "ptaszek"
            # Opcjonalnie wyczyść formularz po sukcesie
            w_first_name.value = ''
            w_last_name.value = ''
            w_pesel.value = ''
            w_dob.value = None
            w_phone.value = ''
            w_email.value = ''
            w_address.value = ''
            w_contact_person.value = ''
            w_blood_group.value = ''
            w_allergies.value = ''
            w_chronic.value = ''
            w_meds.value = ''
            w_notes.value = ''
            w_login.value = ''
            w_password.value = ''
        else:
            # Dodaj klasę CSS do output dla stylizacji błędu
            add_patient_output.add_class("output-error")
            add_patient_output.remove_class("output-success") # Usuń klasę sukcesu jeśli była
            print(f"\u274C {message}") # Czerwony krzyżyk

# Przypisz funkcję callback do przycisku
add_patient_button.on_click(on_add_patient_button_click)

# --- Główny Layout Formularza Dodawania ---
# Główny kontener z klasą CSS i ograniczeniem szerokości
form_container = widgets.VBox([
    custom_css, # Najpierw wstrzykujemy CSS
    widgets.HTML("<h2>Formularz Dodania Nowego Pacjenta</h2>"),
    personal_data_section,
    medical_data_section,
    login_data_section,
    button_container, # Używamy kontenera przycisku
    add_patient_output # Wyświetli komunikaty pod przyciskiem
], layout=widgets.Layout(max_width='800px', margin='20px auto')) # Ustawiamy max-width i auto margin dla centrowania

# Nadajemy głównemu kontenerowi klasę, aby style CSS zadziałały
form_container.add_class('custom-form-container')

# --- Wyświetlenie Formularza ---
print("--- Interfejs Dodawania Pacjenta ---")
# Usunięto informację o '*' bo są teraz częścią etykiet
display(form_container)

--- Interfejs Dodawania Pacjenta ---


In [ ]:
# -*- coding: utf-8 -*-
# Blok 2: Logowanie Pacjenta (Stylizowane ipywidgets)

# --- Importy ---
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML as IPHTML # Używamy aliasu IPHTML
import hashlib
import time # Do symulacji opóźnienia

# --- Konfiguracja Bazy Danych (zakładamy, że istnieje z Bloku 1) ---
DB_NAME = 'medica_plus.db'

def get_db_connection():
    """Nawiązuje połączenie z bazą danych SQLite."""
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

# --- Funkcje Pomocnicze (haszowanie - zakładamy, że istnieje z Bloku 1) ---
def hash_password(password):
    """Proste hashowanie hasła (SHA-256)."""
    return hashlib.sha256(password.encode()).hexdigest()

# --- Logika Backend - Weryfikacja Logowania ---
def verify_login(login, password):
    """Weryfikuje dane logowania i zwraca słownik z danymi pacjenta lub None."""
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute("""
            SELECT id, password_hash, first_name, last_name, email, pesel, dob,
                   phone, address, contact_person, blood_group, allergies,
                   chronic_diseases, current_meds, important_notes, login
            FROM patients
            WHERE login = ?
            """, (login,))
        patient = cursor.fetchone()

        if patient:
            stored_hash = patient['password_hash']
            provided_hash = hash_password(password)
            if stored_hash == provided_hash:
                return dict(patient)
            else:
                return None # Hasło nie pasuje
        else:
            return None # Użytkownik nie istnieje
    except sqlite3.Error as e:
        print(f"Błąd podczas weryfikacji logowania: {e}")
        return None
    finally:
        conn.close()

# --- Zmienna stanu (globalna - dla uproszczenia w Colab) ---
current_patient_data = None

# --- Interfejs Użytkownika (ipywidgets) ze STYLOWANIEM ---

# --- Wstrzykiwanie CSS dla Logowania ---
login_custom_css = widgets.HTML("""
<style>
    /* Główny kontener logowania */
    .custom-login-container {
        background-color: #ffffff;
        padding: 40px 30px;
        border-radius: 8px;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1); /* Replikacja cienia */
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; /* Ustawienie fontu */
        max-width: 400px; /* Ustawienie max-width */
        margin: 20px auto; /* Centrowanie */
        width: 100%; /* Aby max-width działało poprawnie */
    }

    /* Nagłówki w kontenerze logowania */
    .custom-login-container .widget-html h1 {
        font-size: 1.2em;
        color: #555;
        margin-bottom: 10px;
        font-weight: 400;
        text-align: center; /* Wyśrodkowanie tekstu */
    }
    .custom-login-container .widget-html .logo {
        font-size: 2.5em;
        color: #2979FF;
        font-weight: bold;
        margin-bottom: 30px;
        line-height: 1;
        text-align: center; /* Wyśrodkowanie tekstu */
    }

    /* Grupa formularza (etykieta + input) */
    .form-group {
        margin-bottom: 20px;
        text-align: left; /* Etykiety wyrównane do lewej */
    }
    .form-group .widget-label { /* Styl etykiety */
        display: block;
        margin-bottom: 8px;
        font-weight: 600;
        font-size: 0.95em;
        color: #333;
        text-align: left; /* Jawne wyrównanie */
        width: 100%; /* Etykieta na całą szerokość */
    }

    /* Inputy tekstowe i hasła */
    .custom-login-container .widget-text input,
    .custom-login-container .widget-password input {
        width: 100%;
        padding: 12px 15px;
        border: 1px solid #ccc;
        border-radius: 5px;
        font-size: 1em;
        box-sizing: border-box;
        transition: border-color 0.3s ease;
    }
    .custom-login-container .widget-text input:focus,
    .custom-login-container .widget-password input:focus {
        border-color: #2979FF;
        outline: none;
    }

    /* Kontener opcji (Remember me, Forgot password) */
    .login-options-container {
        display: flex !important; /* Używamy !important by nadpisać styl HBox */
        justify-content: space-between !important;
        align-items: center !important;
        margin-bottom: 25px !important;
        font-size: 0.9em;
        width: 100%; /* Aby zajmował całą szerokość */
    }
    .login-options-container .widget-hbox { /* Celowanie w wewnętrzne HBox */
        align-items: center !important; /* Wyrównanie checkboxa i etykiety */
        margin: 0; /* Reset marginesu HBox */
        padding: 0; /* Reset paddingu HBox */
        border: none; /* Reset ramki HBox */
        overflow: visible; /* Upewnienie się, że nic nie jest obcięte */
    }
    .login-options-container .widget-checkbox {
        margin-right: 8px !important; /* Odstęp checkboxa od etykiety */
        margin-left: 0 !important;
        padding: 0 !important;
        min-width: auto; /* Zapobieganie nadmiernemu rozciąganiu */
    }
    .login-options-container .widget-checkbox input[type="checkbox"] {
         accent-color: #2979FF; /* Kolor zaznaczenia */
         margin: 0; /* Reset marginesu samego checkboxa */
    }
    .login-options-container .remember-me-label .widget-label { /* Etykieta 'Zapamiętaj mnie' */
        margin-bottom: 0 !important; /* Usunięcie domyślnego marginesu */
        font-weight: 400 !important;
        color: #555 !important;
        font-size: 1em; /* Reset do rozmiaru bazowego w kontenerze opcji */
    }
    .login-options-container .forgot-password-link a { /* Link 'Zapomniałeś hasła?' */
        color: #2979FF;
        text-decoration: none;
        font-weight: 500;
    }
    .login-options-container .forgot-password-link a:hover {
        text-decoration: underline;
    }

    /* Przycisk logowania */
    .login-button-container .widget-button button {
        background-color: #2979FF !important; /* Używamy !important by nadpisać button_style */
        color: white !important;
        padding: 14px 20px !important;
        border: none !important;
        border-radius: 5px !important;
        font-size: 1.1em !important;
        font-weight: 600 !important;
        cursor: pointer;
        width: 100%; /* Przycisk na całą szerokość */
        transition: background-color 0.3s ease;
        margin-top: 10px; /* Odstęp nad przyciskiem */
    }
    .login-button-container .widget-button button:hover {
        background-color: #1C64F2 !important; /* Ciemniejszy niebieski */
    }

    /* Komunikaty wyjściowe (opcjonalnie - jak poprzednio) */
    .login-output .widget-output {
        margin-top: 15px;
        padding: 10px;
        border-radius: 4px;
        text-align: center; /* Wyśrodkowanie tekstu komunikatu */
    }
    .login-output .widget-output pre {
        font-size: 1em;
        white-space: pre-wrap;
        margin: 0;
    }
    .login-output .output-success { background-color: #e6ffed; border: 1px solid #a3e9a4; color: #1f7c3a; }
    .login-output .output-error { background-color: #ffebee; border: 1px solid #ffcdd2; color: #c62828; }

</style>
""")

# --- Funkcja pomocnicza do tworzenia grupy formularza (Etykieta + Input) ---
def create_login_form_group(widget, label_text):
    widget.description = "" # Czyścimy domyślną etykietę widgetu
    label = widgets.Label(label_text)
    # Zwracamy VBox symulujący .form-group
    return widgets.VBox([label, widget], add_class='form-group')

# --- Widżety Formularza Logowania ---

# Tytuły jako HTML
login_title_html = widgets.HTML("<h1>System Obsługi Pacjentów</h1>")
login_logo_html = widgets.HTML("<div class='logo'>Medica+</div>")

# Pola logowania opakowane w grupy
w_login_username = widgets.Text(placeholder="Wpisz login lub email")
login_username_group = create_login_form_group(w_login_username, "Nazwa użytkownika:")

w_login_password = widgets.Password(placeholder="Wpisz hasło")
login_password_group = create_login_form_group(w_login_password, "Hasło:")

# Opcje "Zapamiętaj mnie" i "Zapomniałeś hasła?"
w_remember_me = widgets.Checkbox(
    value=False,
    indent=False,
    layout=widgets.Layout(width='auto') # Checkbox zajmuje minimalną szerokość
)
remember_me_label = widgets.Label("Zapamiętaj mnie")
remember_me_label.add_class('remember-me-label') # Dodajemy klasę dla stylizacji etykiety
remember_me_box = widgets.HBox(
    [w_remember_me, remember_me_label],
    layout=widgets.Layout(width='auto') # Kontener "Zapamiętaj mnie" zajmuje minimalną szerokość
)

forgot_password_html = widgets.HTML(
    '<a href="#">Zapomniałeś hasła?</a>'
)
forgot_password_link = widgets.Box([forgot_password_html], add_class="forgot-password-link") # Używamy Box zamiast gołego HTML dla łatwiejszego pozycjonowania

# Kontener HBox na opcje z dodaną klasą CSS
options_container = widgets.HBox(
    [remember_me_box, forgot_password_link],
    add_class='login-options-container'
)

# Przycisk Zaloguj w kontenerze dla stylizacji
login_button = widgets.Button(
    description="Zaloguj",
    icon='sign-in',
    tooltip="Kliknij, aby się zalogować",
    # button_style='info' # Usunięte, stylizacja przez CSS
)
# Opakowujemy przycisk w Box, aby zastosować style .login-button-container
login_button_container = widgets.Box([login_button], add_class="login-button-container")


# Pole na komunikaty wyjściowe z klasą CSS
login_output = widgets.Output()
login_output_container = widgets.Box([login_output], add_class="login-output") # Opakowanie dla stylizacji

# --- Logika Przycisku Logowania (Callback) ---
def on_login_button_click(b):
    global current_patient_data
    login_val = w_login_username.value.strip()
    password_val = w_login_password.value

    # Usuwamy poprzednie klasy statusu z output
    login_output_container.remove_class("output-success")
    login_output_container.remove_class("output-error")

    with login_output:
        clear_output(wait=True)

        if not login_val or not password_val:
            login_output_container.add_class("output-error")
            print("\u274C Podaj login i hasło.")
            return

        login_button.disabled = True
        login_button.icon = 'spinner'
        # Symulacja opóźnienia - w praktyce można usunąć time.sleep
        # time.sleep(0.5)

        patient_data_dict = verify_login(login_val, password_val)

        login_button.disabled = False
        login_button.icon = 'sign-in'

        if patient_data_dict:
            current_patient_data = patient_data_dict
            login_output_container.add_class("output-success")
            print(f"\u2705 Zalogowano pomyślnie jako: {current_patient_data['first_name']} {current_patient_data['last_name']}")

            # Czyszczenie pól po udanym logowaniu
            w_login_username.value = ''
            w_login_password.value = ''

            # ----- MIEJSCE NA WYWOŁANIE WIDOKU KARTY PACJENTA -----
            # Tutaj, po pomyślnym logowaniu, zwykle ukrywa się formularz logowania
            # i wyświetla kolejny widok, np. kartę pacjenta.
            # Przykład (odkomentuj, jeśli masz funkcję display_patient_card):
            # clear_output(wait=True)
            # display_patient_card(current_patient_data)
            # ----- KONIEC MIEJSCA NA WYWOŁANIE -----

        else:
            current_patient_data = None
            login_output_container.add_class("output-error")
            print("\u274C Błędny login lub hasło.")


login_button.on_click(on_login_button_click)

# --- Główny Layout Formularza Logowania ---
# Używamy VBox jako głównego kontenera, dodajemy klasę CSS i ustawiamy layout
login_view_container = widgets.VBox([
    login_custom_css, # Wstrzyknięcie stylów CSS na początku
    login_title_html,
    login_logo_html,
    login_username_group, # Grupa z etykietą i inputem
    login_password_group, # Grupa z etykietą i inputem
    options_container,    # Kontener z opcjami
    login_button_container,# Kontener z przyciskiem
    login_output_container # Kontener z polem wyjściowym
], layout=widgets.Layout(align_items='center')) # Centrowanie elementów w VBox

# Nadajemy głównemu kontenerowi klasę dla stylizacji CSS
login_view_container.add_class('custom-login-container')

# --- Wyświetlenie Formularza ---
print("--- Interfejs Logowania Pacjenta (Stylizowany) ---")
display(login_view_container)

--- Interfejs Logowania Pacjenta (Stylizowany) ---


In [ ]:
# -*- coding: utf-8 -*-
# Blok 3: Karta Pacjenta (Stylizowane ipywidgets)

# --- Importy ---
import sqlite3
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML as IPHTML # Używamy aliasu IPHTML
import datetime

# --- Zmienna stanu (globalna) - Zakładamy, że została wypełniona w Bloku 2 ---
# current_patient_data = { ... dane pacjenta ... }

# --- PRZYKŁADOWE DANE DO TESTOWANIA (jeśli Bloki 1 i 2 nie były uruchomione) ---
# Odkomentuj poniższe, jeśli uruchamiasz ten blok niezależnie:
# current_patient_data = {
#      'id': 33966, 'first_name': 'Eleonora', 'last_name': 'Testowa',
#      'pesel': '99010112345', 'dob': '1999-01-01',
#      'phone': '+48 111 222 333', 'email': 'eleonora.t@mail.net',
#      'address': 'ul. Wirtualna 5/10, 01-234 Cyberów',
#      'contact_person': 'Admin Systemu (tel. 123)', 'blood_group': '0 Rh-',
#      'allergies': 'Orzeszki ziemne\nPyłki brzozy',
#      'chronic_diseases': 'Zespół jelita drażliwego',
#      'current_meds': 'Duspatalin Retard (okazjonalnie)\nClatra (wiosna)',
#      'important_notes': 'Pacjentka preferuje kontakt emailowy.',
#      'login': 'eleonora.test'
# }
# --- KONIEC PRZYKŁADOWYCH DANYCH ---


# Symulacja informacji o zalogowanym lekarzu
logged_in_user_name = "Dr. Jan Kowalski" # Przykład

# --- Funkcje Pomocnicze ---
def get_val(data_dict, key, default='-'):
    """Bezpiecznie pobiera wartość ze słownika, zwraca default jeśli brak lub pusta."""
    val = data_dict.get(key, default)
    return val if val not in [None, ''] else default

def format_multiline(text, default='-'):
    """Formatuje tekst wieloliniowy do wyświetlenia w HTML, zachowując nowe linie."""
    if text is None or text == '':
        return default
    # Zastąp znaki nowej linii tagami <br> dla HTML lub użyj <pre>
    # Użycie <pre> jest lepsze dla zachowania formatowania i uniknięcia problemów z CSS
    safe_text = str(text).replace('&', '&').replace('<', '<').replace('>', '>') # Podstawowe escape'owanie HTML
    return f'<pre style="margin: 0; padding: 0; border: none; background: none; font-family: inherit; font-size: inherit; white-space: pre-wrap; word-break: break-word;">{safe_text}</pre>'


# --- Interfejs Użytkownika (ipywidgets) ze STYLOWANIEM ---

# --- Wstrzykiwanie CSS dla Karty Pacjenta ---
patient_card_css = widgets.HTML("""
<style>
    /* Ogólne tło (nie wpłynie na body w Colab/Jupyter, ale ustawiamy dla spójności) */
    .widget-container { /* Ogólny kontener ipywidgets, może być potrzebny */
       /* background-color: #eef2f7; */ /* Tło często jest kontrolowane przez środowisko */
    }

    /* Pasek Nagłówka Aplikacji */
    .custom-header-bar {
        background-color: #0056b3;
        color: white;
        padding: 10px 30px;
        display: flex !important;
        justify-content: space-between !important;
        align-items: center !important;
        width: 100%; /* Pełna szerokość */
        box-sizing: border-box;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .custom-header-bar .logo-html div { /* Celowanie w div wewnątrz HTML */
        font-size: 1.8em;
        font-weight: bold;
    }
    .custom-header-bar .user-info-html div { /* Celowanie w div wewnątrz HTML */
        font-size: 0.9em;
    }
     .custom-header-bar .user-info-html a {
        color: #b3d4fc;
        text-decoration: none;
        margin-left: 15px;
    }
    .custom-header-bar .user-info-html a:hover {
        color: white;
    }

    /* Główna Karta Pacjenta */
    .custom-patient-card {
        background-color: #ffffff;
        border-radius: 8px;
        box-shadow: 0 2px 10px rgba(0, 0, 0, 0.08);
        max-width: 1100px;
        margin: 20px auto !important; /* Centrowanie i margines górny/dolny */
        overflow: hidden;
        border: 1px solid #eee; /* Alternatywa dla cienia, jeśli cień nie działa */
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }

    /* Nagłówek Karty */
    .custom-card-header {
        padding: 15px 25px;
        border-bottom: 1px solid #eeeeee;
        display: flex !important;
        justify-content: space-between !important;
        align-items: center !important;
    }
    .custom-card-header .card-title-html h2 { /* Celowanie w h2 */
        margin: 0;
        font-size: 1.4em;
        color: #333;
        font-weight: 600;
    }
    .custom-card-header .patient-id-html span { /* Celowanie w span */
        font-size: 0.9em;
        font-weight: 500;
        color: #ffffff;
        background-color: #6c757d;
        padding: 3px 8px;
        border-radius: 4px;
    }

    /* Ciało Karty (Grid) */
    .custom-card-body {
        padding: 25px !important;
        /* display: grid; */ /* GridBox sam tworzy grid */
        /* grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); */ /* Definiowane w GridBox layout */
        /* gap: 25px; */ /* Definiowane w GridBox layout */
    }

    /* Sekcja Danych */
    .custom-data-section {
        background-color: #f8f9fa !important;
        padding: 20px !important;
        border-radius: 5px !important;
        border: 1px solid #e9ecef !important;
        box-sizing: border-box;
        /* Domyślnie zajmuje jedną komórkę gridu */
    }
     /* Sekcja na całą szerokość */
    .custom-data-section.full-width {
         /* grid_column jest ustawiany w layoucie widgetu */
    }
    .custom-data-section .section-content-html h3 { /* Celowanie w h3 */
        color: #0056b3;
        font-size: 1.1em;
        margin-top: 0;
        margin-bottom: 15px;
        padding-bottom: 8px;
        border-bottom: 1px solid #dee2e6;
        font-weight: 600;
    }
    .custom-data-section .section-content-html .data-item { /* Styl dla div.data-item */
        margin-bottom: 10px;
        font-size: 0.95em;
        line-height: 1.5;
        color: #495057;
    }
    .custom-data-section .section-content-html .data-item strong { /* Styl dla etykiety strong */
        display: inline-block;
        min-width: 140px; /* Zwiększono min-width dla lepszego wyrównania */
        color: #343a40;
        font-weight: 600;
        vertical-align: top; /* Lepsze wyrównanie z wieloliniowymi wartościami */
    }
     /* Styl dla pre (wartości wieloliniowe) */
     .custom-data-section .section-content-html .data-item pre {
        display: inline-block; /* Aby był w linii z etykietą */
        margin: 0;
        padding: 0;
        border: none;
        background: none;
        font-family: inherit;
        font-size: inherit;
        white-space: pre-wrap;
        word-break: break-word;
        vertical-align: top; /* Wyrównanie do góry */
        /* Dodatkowa szerokość, aby uniknąć zbyt wczesnego zawijania */
        max-width: calc(100% - 150px); /* Przykładowo: 100% minus szerokość etykiety + margines */
    }
    /* Historia Wizyt */
    .custom-data-section .history-list-html p {
         margin: 5px 0;
         font-size: 0.95em;
         color: #495057;
    }

    /* Stopka z Akcjami */
    .custom-card-actions {
        background-color: #f8f9fa !important;
        padding: 15px 25px !important;
        border-top: 1px solid #eeeeee !important;
        display: flex !important;
        justify-content: flex-end !important; /* Wyrównanie do prawej */
        flex-wrap: wrap; /* Zawijanie przycisków na mniejszych ekranach */
    }
    /* Przyciski Akcji */
    .custom-card-actions .widget-button { /* Celujemy w kontener przycisku */
        margin-left: 10px !important;
        margin-top: 5px !important; /* Dodano margines górny dla zawijania */
        margin-bottom: 5px !important; /* Dodano margines dolny dla zawijania */
    }
    .custom-card-actions .widget-button button { /* Celujemy w sam przycisk */
        background-color: #0056b3 !important; /* Nadpisujemy button_style */
        color: white !important;
        padding: 10px 20px !important;
        border: none !important;
        border-radius: 5px !important;
        font-size: 0.9em !important;
        font-weight: 500 !important;
        cursor: pointer;
        transition: background-color 0.3s ease;
    }
    .custom-card-actions .widget-button button:hover {
        background-color: #004494 !important;
    }
     /* Opcjonalne: Nadpisanie kolorów dla specyficznych przycisków (jeśli button_style nie wystarczy) */
     .custom-card-actions .edit-button button { background-color: #007bff !important; } /* Example: Edit blue */
     .custom-card-actions .edit-button button:hover { background-color: #0056b3 !important; }
     .custom-card-actions .add-visit-button button { background-color: #17a2b8 !important; } /* Example: Visit cyan */
     .custom-card-actions .add-visit-button button:hover { background-color: #117a8b !important; }
     .custom-card-actions .order-test-button button { background-color: #ffc107 !important; color: #212529 !important; } /* Example: Test yellow */
     .custom-card-actions .order-test-button button:hover { background-color: #e0a800 !important; }
     .custom-card-actions .add-prescription-button button { background-color: #28a745 !important; } /* Example: Prescription green */
     .custom-card-actions .add-prescription-button button:hover { background-color: #1e7e34 !important; }

</style>
""")

# --- Widżety Karty Pacjenta ---

# 1. Pasek Nagłówka Aplikacji
header_logo_html = widgets.HTML(f"<div class='logo'>Medica+</div>")
header_user_info_html = widgets.HTML(f"""
    <div>
        Zalogowany jako: <strong>{logged_in_user_name}</strong>
        <a href='#' onclick='alert("Wylogowywanie niezaimplementowane."); return false;'>Wyloguj</a>
    </div>
""")
# Dodajemy klasy CSS do kontenerów HTML dla łatwiejszego targetowania
header_logo = widgets.Box([header_logo_html], add_class="logo-html")
header_user_info = widgets.Box([header_user_info_html], add_class="user-info-html")

header_bar = widgets.HBox([header_logo, header_user_info])
header_bar.add_class("custom-header-bar") # Dodanie klasy CSS do HBox

# 2. Główny Kontener Karty (VBox)
patient_card_container = widgets.VBox()
patient_card_container.add_class("custom-patient-card")

# 3. Nagłówek Karty (HBox wewnątrz kontenera)
card_title_html = widgets.HTML() # Wypełnimy dynamicznie
patient_id_html = widgets.HTML() # Wypełnimy dynamicznie
card_title = widgets.Box([card_title_html], add_class="card-title-html")
patient_id_widget = widgets.Box([patient_id_html], add_class="patient-id-html")

card_header = widgets.HBox([card_title, patient_id_widget])
card_header.add_class("custom-card-header")

# 4. Sekcje Danych (VBoxy z wewnętrznym HTML)
personal_data_content_html = widgets.HTML()
personal_data_section = widgets.VBox(
    [widgets.Box([personal_data_content_html], add_class="section-content-html")], # Opakowanie HTML w Box
    layout=widgets.Layout(grid_column='span 1') # Domyślnie zajmuje 1 kolumnę
)
personal_data_section.add_class("custom-data-section")

medical_data_content_html = widgets.HTML()
medical_data_section = widgets.VBox(
    [widgets.Box([medical_data_content_html], add_class="section-content-html")],
    layout=widgets.Layout(grid_column='span 1')
)
medical_data_section.add_class("custom-data-section")

history_content_html = widgets.HTML() # Na razie statyczna
history_data_section = widgets.VBox(
    [widgets.Box([history_content_html], add_class="history-list-html section-content-html")], # Dwie klasy
    layout=widgets.Layout(grid_column='1 / -1') # Ustawienie rozpięcia na całą szerokość
)
history_data_section.add_class("custom-data-section full-width") # Dwie klasy dla stylizacji i logiki

# 5. Ciało Karty (GridBox)
card_body = widgets.GridBox(
    children=[personal_data_section, medical_data_section, history_data_section],
    layout=widgets.Layout(
        padding='25px',
        grid_template_columns='repeat(auto-fit, minmax(300px, 1fr))',
        gap='25px',
        width='100%' # Grid zajmuje całą szerokość kontenera karty
    )
)
# card_body.add_class("custom-card-body") # Klasa dodana do GridBox dla ew. stylizacji samego gridu

# 6. Przyciski Akcji (HBox)
# Używamy klas CSS do stylizacji kolorów zamiast button_style
edit_button = widgets.Button(description="Edytuj Dane Pacjenta", icon='edit', disabled=False, tooltip="Edytuj dane osobowe i medyczne pacjenta")
edit_button.add_class("edit-button")

add_visit_button = widgets.Button(description="Dodaj Nową Wizytę", icon='calendar-plus-o', disabled=False, tooltip="Zarejestruj nową wizytę dla pacjenta")
add_visit_button.add_class("add-visit-button")

order_test_button = widgets.Button(description="Zleć Badanie", icon='flask', disabled=False, tooltip="Wystaw skierowanie na badania")
order_test_button.add_class("order-test-button")

add_prescription_button = widgets.Button(description="Wystaw Receptę", icon='medkit', disabled=False, tooltip="Wystaw e-receptę")
add_prescription_button.add_class("add-prescription-button")

action_buttons = widgets.HBox(
    [edit_button, add_visit_button, order_test_button, add_prescription_button],
    layout=widgets.Layout(justify_content='flex-end', flex_wrap='wrap') # Wyrównanie i zawijanie
)
action_buttons.add_class("custom-card-actions")

# --- Logika Dynamicznego Wypełniania Karty ---
def display_patient_card(patient_data):
    """Wypełnia widżety karty danymi pacjenta i wyświetla je."""
    if not patient_data or not isinstance(patient_data, dict):
        clear_output(wait=True)
        error_html = widgets.HTML("<h2 style='color: red; text-align: center; padding: 40px;'>Błąd: Brak danych pacjenta do wyświetlenia.</h2><p style='text-align:center;'>Upewnij się, że pacjent został poprawnie zalogowany lub dane testowe są aktywne.</p>")
        display(error_html)
        return

    # --- Wypełnianie Nagłówka Karty ---
    card_title_html.value = f"<h2>Karta Pacjenta: - {get_val(patient_data, 'first_name')} {get_val(patient_data, 'last_name')}</h2>"
    patient_id_html.value = f"<span>ID Pacjenta: {get_val(patient_data, 'id')}</span>"

    # --- Wypełnianie Sekcji Danych Osobowych ---
    personal_html_content = f"""
        <h3>Dane Osobowe</h3>
        <div class="data-item"><strong>Data urodzenia:</strong> {get_val(patient_data, 'dob')}</div>
        <div class="data-item"><strong>PESEL:</strong> {get_val(patient_data, 'pesel')}</div>
        <div class="data-item"><strong>Adres:</strong> {get_val(patient_data, 'address')}</div>
        <div class="data-item"><strong>Telefon:</strong> {get_val(patient_data, 'phone')}</div>
        <div class="data-item"><strong>Email:</strong> {get_val(patient_data, 'email')}</div>
        <div class="data-item"><strong>Osoba kontaktowa:</strong> {get_val(patient_data, 'contact_person')}</div>
    """
    personal_data_content_html.value = personal_html_content

    # --- Wypełnianie Sekcji Danych Medycznych (z formatowaniem wieloliniowym) ---
    medical_html_content = f"""
        <h3>Podstawowe Informacje Medyczne</h3>
        <div class="data-item"><strong>Grupa krwi:</strong> {get_val(patient_data, 'blood_group')}</div>
        <div class="data-item"><strong>Alergie:</strong> {format_multiline(patient_data.get('allergies'))}</div>
        <div class="data-item"><strong>Choroby przewlekłe:</strong> {format_multiline(patient_data.get('chronic_diseases'))}</div>
        <div class="data-item"><strong>Stałe leki:</strong> {format_multiline(patient_data.get('current_meds'))}</div>
        <div class="data-item"><strong>Ważne uwagi:</strong> {format_multiline(patient_data.get('important_notes'))}</div>
    """
    medical_data_content_html.value = medical_html_content

    # --- Wypełnianie Sekcji Historii (na razie statyczna) ---
    history_content = """
        <h3>Historia Wizyt i Zdarzeń Medycznych (Przykład)</h3>
        <p>20.05.2024 - Wizyta kontrolna, Kardiologia, Dr. Kowalski</p>
        <p>10.02.2024 - Badanie EKG (zlecone)</p>
        <p>15.11.2023 - Wizyta pierwszorazowa, Lekarz Rodzinny, Dr. Nowak</p>
        <p>05.09.2023 - Wyniki badań laboratoryjnych (odebrane)</p>
        <p>...</p>
        <p><em style="font-size:0.9em; color:#777;">(Dynamiczne ładowanie historii niezaimplementowane w tym przykładzie)</em></p>
    """
    history_content_html.value = history_content

    # --- Składanie finalnego widoku ---
    patient_card_container.children = [card_header, card_body, action_buttons]

    # Główny kontener widoku (CSS + Header + Karta)
    full_view = widgets.VBox([
        patient_card_css, # Wstrzyknięcie CSS
        header_bar,
        patient_card_container
    ])

    # Wyczyszczenie poprzedniego outputu i wyświetlenie nowego widoku
    clear_output(wait=True)
    print("--- Karta Pacjenta (Stylizowana) ---") # Nagłówek informacyjny
    display(full_view)

# --- Główny punkt wejścia dla tego bloku ---

# Sprawdź, czy dane pacjenta istnieją (z poprzedniego bloku lub testowe)
if 'current_patient_data' in locals() and current_patient_data:
     # Wywołaj funkcję, która wypełni i wyświetli kartę
     display_patient_card(current_patient_data)
else:
     # Wyświetl komunikat, jeśli dane pacjenta nie są dostępne
     clear_output(wait=True) # Wyczyść ewentualne poprzednie komunikaty
     display(widgets.HTML("""
         <div style='background-color: #fff3cd; border: 1px solid #ffeeba; color: #856404; padding: 20px; margin: 20px auto; max-width: 800px; border-radius: 5px; text-align: center;'>
             <h2>Brak Aktywnego Pacjenta</h2>
             <p>Nie znaleziono danych zalogowanego pacjenta (zmienna <code>current_patient_data</code> jest pusta).</p>
             <p>Proszę najpierw uruchomić <strong>Blok 2: Logowanie Pacjenta</strong> i pomyślnie się zalogować, LUB odkomentować przykładowe dane testowe w kodzie <strong>Bloku 3</strong>.</p>
         </div>
     """))

# Opcjonalnie: Przycisk Wyloguj (symulacja powrotu do ekranu logowania)
# logout_button = widgets.Button(description="Wyloguj (Powrót do Logowania)", button_style='danger', icon='sign-out', layout=widgets.Layout(margin='20px auto', width='auto'))
# def on_logout_click(b):
#      global current_patient_data
#      current_patient_data = None
#      clear_output(wait=True)
#      # Tutaj w realnej aplikacji wywołalibyśmy funkcję wyświetlającą widok logowania
#      print("Wylogowano. Aby kontynuować, uruchom ponownie Blok 2: Logowanie.")
#      # display_login_view() # Przykładowe wywołanie
# logout_button.on_click(on_logout_click)

# Jeśli dane pacjenta istnieją, pokaż przycisk wylogowania pod kartą
# if 'current_patient_data' in locals() and current_patient_data:
#      display(logout_button)

--- Karta Pacjenta (Stylizowana) ---
